In [1]:
import pandas as pd
import ast

In [2]:
ob_products_dir = "orangebook/products.txt"
ob_products = pd.read_csv(ob_products_dir, sep="~")
ob_patent_dir = "orangebook/patent.txt"
ob_patent = pd.read_csv(ob_patent_dir, sep="~")

In [ ]:
ob_products.Product_No.nunique()

In [ ]:
ob_products.Ingredient.nunique()

In [ ]:
ob_products.shape

In [ ]:
ob_products.loc[ob_products.Product_No==1]

In [ ]:
ob_patent

In [ ]:
ob_patent.Patent_No.nunique()

In [ ]:
ob_patent.Patent_No.nunique()

In [ ]:
len(set(zip(ob_patent['Patent_No'],ob_patent['Product_No'], ob_patent["Appl_No"])))


In [1]:
ob_query = """
#standardSQL
SELECT publication_number, 
ANY_VALUE(filing_year) AS filing_year, 
ANY_VALUE(assignee) AS assignee, 
ARRAY_AGG(DISTINCT applicant_full_name) AS applicant, 
ARRAY_AGG(DISTINCT trade_name) AS trade_name,
ARRAY_AGG(DISTINCT pref_name) AS target, 
ARRAY_AGG(DISTINCT mesh_heading) AS mesh_heading
FROM (
  SELECT pref_name, mechanism_of_action, dm.record_id,
  di.mesh_heading,
  prod.trade_name, 
  prod.route, 
  prod.applicant_full_name, 
  prod.approval_date, 
  cm.publication_number,
  pat.patent_use_code, 
  pat.drug_product_flag, 
  pat.drug_substance_flag, 
  pubs.cpc,
  CAST(FLOOR(filing_date / 10000) AS INT64) AS filing_year,
  (SELECT ARRAY_AGG(a.name) FROM UNNEST(pubs.assignee_harmonized) AS a) AS assignee
  FROM `patents-public-data.ebi_chembl.drug_mechanism_23` AS dm
  JOIN `patents-public-data.ebi_chembl.drug_indication_23` AS di ON di.record_id = dm.record_id
  JOIN `patents-public-data.ebi_chembl.target_dictionary_23` AS td ON td.tid = dm.tid
  JOIN `patents-public-data.ebi_chembl.formulations_23` AS form ON form.record_id = dm.record_id
  JOIN `patents-public-data.ebi_chembl.products_23` AS prod ON prod.product_id = form.product_id
  JOIN `patents-public-data.ebi_chembl.product_patents_23` AS pat ON pat.product_id = prod.product_id
  JOIN `patents-public-data.ebi_chembl.match_23` AS cm ON cm.patent_no = pat.patent_no
  JOIN `patents-public-data.patents.publications` AS pubs ON pubs.publication_number = cm.publication_number
)
GROUP BY publication_number
ORDER BY filing_year


"""

In [2]:
from pandas.io import gbq
import pandas as pd

bq_project = "thesis-jaeyoon"

In [4]:
ob_df = gbq.read_gbq(
            query=ob_query,
            project_id=bq_project,
            verbose=True,
            dialect='standard')

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/pandas/io/gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [5]:
ob_df = pd.read_csv("orangebook_chembl.csv", sep='\t')

In [15]:
(ast.literal_eval(ob_df.iloc[3254]["mesh_heading"]))

['PAIN', 'OPIOID-RELATED DISORDERS']

In [17]:
disease_year = pd.DataFrame(columns=["publication_number","filing_year", "assignee","indication"])
for i, row in ob_df.iterrows():
    headings = ast.literal_eval(row["mesh_heading"])
    new_row = dict()
    new_row["publication_number"] = row["publication_number"]
    new_row["filing_year"] = row["filing_year"]
    new_row["assignee"] = '|'.join(ast.literal_eval(row["assignee"])).upper()
    
    for h in headings:
        new_row["indication"] = h
        disease_year = disease_year.append(new_row, ignore_index=True)

In [19]:
disease_year.to_csv("orangebook_patent_indicaton.csv", sep='\t', index=False)

In [20]:
by_disease_year = disease_year.groupby(["filing_year", "indication"])["publication_number"].count()

In [21]:
by_disease_year = by_disease_year.reset_index()

In [69]:
disease = "gastroenteritis".upper()

In [70]:
by_disease_year.loc[by_disease_year.indication.str.contains(disease)]

,filing_year,indication,publication_number
1033,1997,GASTROENTERITIS,2
1400,1998,GASTROENTERITIS,1
5114,2007,GASTROENTERITIS,1
8584,2015,GASTROENTERITIS,1
